## Data_stats_check

In [3]:
import pandas as pd
train = pd.read_csv('/home/shan/Desktop/netlab/Esophagitis/Proper_split_data/train.csv')
dev = pd.read_csv('/home/shan/Desktop/netlab/Esophagitis/Proper_split_data/dev.csv')
test = pd.read_csv('/home/shan/Desktop/netlab/Esophagitis/Proper_split_data/test.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PMRN             1243 non-null   int64  
 1   Free Text Grade  1243 non-null   float64
 2   Full Text        1243 non-null   object 
 3   TxDose           1243 non-null   float64
 4   TxFx             1243 non-null   int64  
 5   esophv55         1243 non-null   float64
 6   esophmean        1243 non-null   float64
 7   technique        1243 non-null   object 
 8   frequency        1243 non-null   object 
 9   ih               1243 non-null   object 
 10  ap               1243 non-null   object 
 11  sec_text         1243 non-null   object 
 12  exam             1243 non-null   object 
 13  rot              1243 non-null   object 
 14  ros              1243 non-null   object 
 15  TumorEQD2        1243 non-null   float64
 16  Age              1243 non-null   int64  
 17  Gender        

In [ ]:
test = pd.concat([dev, test])

In [5]:
'esophagitis' in train['Full Text'][0].lower()

False

In [ ]:
train_check_g = []
train_check_tf = []
for grade, text in zip(train['Free Text Grade'], train['Full Text']):
	c = 'esophagitis' in text.lower()
	if c:
		c = 1
	else:
		c = 0
	train_check_g.append(grade)
	train_check_tf.append(c)

In [ ]:
test_check_tt = []
for grade, text in zip(test['Free Text Grade'], test['Full Text']):
	c = str('esophagitis' in text.lower())
	test_check_tt.append(str(grade)+c)

In [75]:
train_check_tt = []
for grade, text in zip(train['Free Text Grade'], train['Full Text']):
	c = str('esophagitis' in text.lower())
	train_check_tt.append(str(grade)+c)

In [82]:
from collections import Counter
Counter(train_check_tt)

Counter({'2.0False': 9,
         '1.0True': 138,
         '1.0False': 8,
         '2.0True': 127,
         '0.0False': 550,
         '0.0True': 319,
         '3.0True': 74,
         '3.0False': 18})

In [83]:
Counter(test_check_tt)

Counter({'1.0True': 52,
         '1.0False': 9,
         '2.0True': 48,
         '3.0True': 8,
         '0.0True': 73,
         '0.0False': 88,
         '2.0False': 3})

In [ ]:
test_check_g = []
test_check_tf = []
for grade, text in zip(test['Free Text Grade'], test['Full Text']):
	c = 'esophagitis' in text.lower()
	if c:
		c = 1
	else:
		c = 0
	test_check_g.append(grade)
	test_check_tf.append(c)

In [68]:
train_check = pd.DataFrame({
	'grade':train_check_g,
	'Eso_is_in_text':train_check_tf
})

In [69]:
test_check = pd.DataFrame({
	'grade':test_check_g,
	'Eso_is_in_text':test_check_tf
})

In [71]:
train_check.to_csv('/home/shan/Desktop/netlab/Esophagitis/baselines/stats/t_check.csv', index=False)
test_check.to_csv('/home/shan/Desktop/netlab/Esophagitis/baselines/stats/test_check.csv', index=False)

## Data preprocessing, extracting entities

In [2]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple") # pass device=0 if using gpu


/home/shan/miniconda3/envs/l8/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
PATH1 = '/home/shan/Desktop/netlab/Esophagitis/aug_data_v3/'
PATH2 = '/home/shan/Desktop/netlab/Esophagitis/Proper_split_data/'

In [28]:
import pandas as pd
train = pd.read_csv(PATH1+'train.csv')
dev = pd.read_csv(PATH1+'dev.csv')
test = pd.read_csv(PATH1+'test.csv')

In [16]:
train = pd.read_csv(PATH2+'train.csv')
dev = pd.read_csv(PATH2+'dev.csv')
test = pd.read_csv(PATH2+'test.csv')

In [21]:
def add_entities(df):
	'''
	input a dataframe and add a column of extracted entities from the pipeline
	'''
	result = []
	for i in df['Full Text']:
		result.append(' '.join([ent['word'] for ent in pipe(i)]))
	return result

In [22]:
# inference time about 11mins (for auged data, 4mins for non-auged data)
train_r = add_entities(train)
dev_r = add_entities(dev)
test_r = add_entities(test)

In [30]:
train['ents'] = train_r
dev['ents'] = dev_r
test['ents'] = test_r

In [29]:
# test = pd.concat([dev, test])
#change labeling name
try:
	train['degree'] = train['Free Text Grade']
	test['degree'] = test['Free Text Grade']
except:
	print('using augmented data, do not need to do this')

using augmented data, do not need to do this


In [30]:
## adding binary class labels
train['binary'] = [1 if int(i)>0 else 0 for i in train['degree']]
test['binary'] = [1 if int(i)>0 else 0 for i in test['degree']] 

train['severe'] = [1 if int(i)>1 else 0 for i in train['degree']]
test['severe'] = [1 if int(i)>1 else 0 for i in test['degree']] 

In [8]:
# filter out 0s and do grade1 vs grade2&3
# train = train.loc[train['degree'] >= 1]
# test = test.loc[test['degree'] >= 1]

train['severe'] = [1 if int(i)>1 else 0 for i in train['degree']]
test['severe'] = [1 if int(i)>1 else 0 for i in test['degree']] 

## Baseline 1, BoW on full text

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
  
count_vect = CountVectorizer(ngram_range=(1,1), # to use bigrams ngram_range=(2,2)
                           stop_words='english')

X_train_counts = count_vect.fit_transform(train['Full Text'])

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(3663, 22942)

In [32]:
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [33]:
from sklearn import metrics
text_clf.fit(train['Full Text'], train.binary)
predicted = text_clf.predict(test['Full Text'])
print(metrics.classification_report(test.binary, predicted))

              precision    recall  f1-score   support

           0       0.65      0.99      0.78        85
           1       0.86      0.12      0.20        52

    accuracy                           0.66       137
   macro avg       0.75      0.55      0.49       137
weighted avg       0.73      0.66      0.56       137



In [34]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
])

In [35]:
from sklearn import metrics
text_clf.fit(train['Full Text'], train.binary)
predicted = text_clf.predict(test['Full Text'])
print(metrics.classification_report(test.binary, predicted))

              precision    recall  f1-score   support

           0       0.75      0.93      0.83        85
           1       0.81      0.50      0.62        52

    accuracy                           0.77       137
   macro avg       0.78      0.71      0.73       137
weighted avg       0.78      0.77      0.75       137



In [36]:
from sklearn import metrics
text_clf.fit(train['Full Text'], train.severe)
predicted = text_clf.predict(test['Full Text'])
print(metrics.classification_report(test.severe, predicted))

              precision    recall  f1-score   support

           0       0.82      0.99      0.90       109
           1       0.83      0.18      0.29        28

    accuracy                           0.82       137
   macro avg       0.83      0.58      0.60       137
weighted avg       0.83      0.82      0.78       137



## Baseline2 BoW on entities

In [46]:
from sklearn import metrics
text_clf.fit(train['ents'], train.binary)
predicted = text_clf.predict(test['ents'])
print(metrics.classification_report(test.binary, predicted))

              precision    recall  f1-score   support

           0       0.68      0.97      0.80       161
           1       0.90      0.39      0.55       120

    accuracy                           0.72       281
   macro avg       0.79      0.68      0.67       281
weighted avg       0.78      0.72      0.69       281



In [48]:
from sklearn import metrics
text_clf.fit(train['ents'], train.severe)
predicted = text_clf.predict(test['ents'])
print(metrics.classification_report(test.severe, predicted))

              precision    recall  f1-score   support

           0       0.80      0.99      0.88       222
           1       0.50      0.05      0.09        59

    accuracy                           0.79       281
   macro avg       0.65      0.52      0.49       281
weighted avg       0.73      0.79      0.72       281



## Baseline3 BoW on Assement Plan

In [25]:
from sklearn import metrics
text_clf.fit(train['ap'], train.binary)
predicted = text_clf.predict(test['ap'])
print(metrics.classification_report(test.binary, predicted))

              precision    recall  f1-score   support

           0       0.70      0.92      0.80        85
           1       0.73      0.37      0.49        52

    accuracy                           0.71       137
   macro avg       0.72      0.64      0.64       137
weighted avg       0.71      0.71      0.68       137



In [26]:
from sklearn import metrics
text_clf.fit(train['ap'], train.severe)
predicted = text_clf.predict(test['ap'])
print(metrics.classification_report(test.severe, predicted))

              precision    recall  f1-score   support

           0       0.81      0.99      0.89       109
           1       0.67      0.07      0.13        28

    accuracy                           0.80       137
   macro avg       0.74      0.53      0.51       137
weighted avg       0.78      0.80      0.73       137

